In [18]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
import xgboost as xgb
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.preprocessing import StandardScaler
import joblib

In [19]:
df = pd.read_csv('../../data/training_data.csv', sep=',')
df.drop(columns=['Rating Average'],inplace=True)
df

,Min Players,Max Players,Play Time,Min Age,Users Rated,Complexity Average,Owned Users,Abstract Games,Children's Games,Customizable Games,...,Wargames,Social Interaction,Strategy and Planning,Conflict Resolution,Exploration,Randomness and Luck,Resource Management,Puzzle Solving,Miscellaneous / Other,Collaboration
0,1,4,120.000000,14,42055,3.8604,68323.0,0,0,0,...,0,0,1,0,1,0,1,1,0,1
1,2,4,60.000000,13,41643,2.8405,65294.0,0,0,0,...,0,0,1,0,1,0,1,1,0,1
2,2,4,120.000000,14,19217,3.9129,28785.0,0,0,0,...,0,0,1,0,0,0,1,0,0,0
3,1,5,120.000000,12,64864,3.2406,87099.0,0,0,0,...,0,0,1,1,0,0,1,1,0,0
4,3,6,431.858995,14,13468,4.2219,16831.0,0,0,0,...,0,1,1,1,0,1,1,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20315,2,2,30.000000,4,1340,1.0000,427.0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
20316,2,99,60.000000,5,2154,1.0455,1533.0,0,0,0,...,0,0,0,0,0,1,0,1,0,0
20317,2,4,30.000000,3,4006,1.0779,5788.0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
20318,2,6,30.000000,3,3783,1.0201,4400.0,0,1,0,...,0,0,0,0,0,1,0,0,0,0


In [20]:
X = df.drop(columns=['Complexity Average'])
y = df['Complexity Average']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

pipeline_xgb = Pipeline([
    ('scaler', StandardScaler()),
    ('xgb', xgb.XGBRegressor(random_state=42))
])

In [21]:
param_grid = {
    'xgb__n_estimators': [50, 100, 200],
    'xgb__learning_rate': [0.01, 0.05, 0.1],
    'xgb__max_depth': [3, 5, 7],
    'xgb__subsample': [0.7, 0.8, 0.9],
    'xgb__colsample_bytree': [0.7, 0.8, 0.9],
    'xgb__gamma': [0, 0.1, 0.2]
}

# Set up GridSearchCV with 5-fold cross-validation
grid_search = GridSearchCV(estimator=pipeline_xgb, param_grid=param_grid,
                           cv=5, scoring='neg_mean_squared_error',
                           n_jobs=-1, verbose=2)

# Fit the model
grid_search.fit(X_train, y_train)

# Best parameters and best score
print("Best parameters found: ", grid_search.best_params_)
print("Best score (Negative MSE): ", grid_search.best_score_)

D:\alpay\School\J3\IP\backend-ai-service\venv\Lib\site-packages\sklearn\utils\_tags.py:354: FutureWarning: The XGBRegressor or classes from which it inherits use `_get_tags` and `_more_tags`. Please define the `__sklearn_tags__` method, or inherit from `sklearn.base.BaseEstimator` and/or other appropriate mixins such as `sklearn.base.TransformerMixin`, `sklearn.base.ClassifierMixin`, `sklearn.base.RegressorMixin`, and `sklearn.base.OutlierMixin`. From scikit-learn 1.7, not defining `__sklearn_tags__` will raise an error.
  warnings.warn(


Fitting 5 folds for each of 729 candidates, totalling 3645 fits
Best parameters found:  {'xgb__colsample_bytree': 0.7, 'xgb__gamma': 0, 'xgb__learning_rate': 0.01, 'xgb__max_depth': 3, 'xgb__n_estimators': 50, 'xgb__subsample': 0.7}
Best score (Negative MSE):  nan


D:\alpay\School\J3\IP\backend-ai-service\venv\Lib\site-packages\sklearn\model_selection\_search.py:1107: UserWarning: One or more of the test scores are non-finite: [nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan

In [22]:
y_pred = grid_search.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)

print(f"Mean Squared Error: {mse}")
print(f"Mean Absolute Error: {mae}")
print(f"R² Score: {r2}")

D:\alpay\School\J3\IP\backend-ai-service\venv\Lib\site-packages\sklearn\utils\_tags.py:354: FutureWarning: The XGBRegressor or classes from which it inherits use `_get_tags` and `_more_tags`. Please define the `__sklearn_tags__` method, or inherit from `sklearn.base.BaseEstimator` and/or other appropriate mixins such as `sklearn.base.TransformerMixin`, `sklearn.base.ClassifierMixin`, `sklearn.base.RegressorMixin`, and `sklearn.base.OutlierMixin`. From scikit-learn 1.7, not defining `__sklearn_tags__` will raise an error.
  warnings.warn(
D:\alpay\School\J3\IP\backend-ai-service\venv\Lib\site-packages\sklearn\utils\_tags.py:354: FutureWarning: The XGBRegressor or classes from which it inherits use `_get_tags` and `_more_tags`. Please define the `__sklearn_tags__` method, or inherit from `sklearn.base.BaseEstimator` and/or other appropriate mixins such as `sklearn.base.TransformerMixin`, `sklearn.base.ClassifierMixin`, `sklearn.base.RegressorMixin`, and `sklearn.base.OutlierMixin`. From 

AttributeError: 'super' object has no attribute '__sklearn_tags__'

In [13]:
joblib.dump(grid_search.best_estimator_, '../../models/complexity_boost_regression.pkl')

['../../models/complexity_boost_regression.pkl']